In [2]:
print("Importing packages...")
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.linalg as la

sys.path.insert(1, 'C:/Users/31676/Desktop/research/pde/Python code/GPPDE_v0/src')
from hydro import *
from gp import *
from funcs import *
from richards import picard_solver
from viz import plot_3d
print("Importing packages...Done")

np.set_printoptions(precision=4)
np.set_printoptions(threshold=np.inf)

thta = np.loadtxt('sol.txt')
print("theta dimension", thta.shape)

dz = -0.005
z = np.arange(-0.01,-0.3+dz,dz)

dt = 86400
t = np.arange(dt, dt * 91, dt)

noise_pct = 0.01
X = np.dstack(np.meshgrid(z, t)).reshape(-1, 2)
y = thta.T.flatten()
y += noise_pct * np.std(y) * np.random.randn(len(y))
h_para = np.loadtxt('para.txt', skiprows = 1)

ny = len(y)
print(ny)
bound_ind = np.array([])
for i in range(ny):
    if(abs(X[i, 0]) <= 0.274) and (abs(X[i, 0]) >= 0.026) and (X[i, 1] > 86400 * 4):
        bound_ind = np.append(bound_ind, i)
print("length of sel", len(bound_ind))
bound_ind = bound_ind[np.arange(0, len(bound_ind), 10)]
bad = bound_ind.astype(int)
X = X[bad, :]
y = y[bad]
    
print("X shape", X.shape)
print("y length", len(y))

ny = len(y)
fix_boundary = np.full(ny, 0)
y_mean = np.mean(y, axis=0)
y_std = np.std(y, axis=0)
prior_mean = np.full(ny, y_mean)

diff_z = np.subtract.outer(X[:,0], X[:,0])
diff_t = np.subtract.outer(X[:,1], X[:,1])

# hyperparameter estimation
def obj_func(theta):
    param = np.exp(theta)
    Ky, gradient = Ky_and_derivs(diff_z, diff_t, param, fix_boundary)
    log_like, d_param = log_like_and_derivs(y, prior_mean, Ky, gradient)
    return -log_like, -d_param

# initial value
l_z = np.std(X[:, 0])
l_t = np.std(X[:, 1])
sigma_s2 = np.std(y) / np.sqrt(2)
sigma_y2 = np.std(y) / np.sqrt(2)

param = np.array([sigma_s2, l_z, l_t, sigma_y2])
param_range = np.array([
        [1e-4, 1],
        [0.01, 100],
        [0.01, 10000000],
        [1e-9, 1]
    ])

log_like, d_param = obj_func(np.log(param))
print("initial log_like =", log_like)
print("initial deri=", d_param)

if 1:
    opt_res = minimize(obj_func, 
        np.log(param), 
        method="L-BFGS-B", 
        jac=True, 
        bounds=np.log(param_range)
        ,callback=show_progress
    )
    param_opt = np.exp(opt_res.x)
    func_max = -opt_res.fun
    print("opt para =", param_opt)
    print("obj =", func_max)
    param = param_opt.copy()
    
post_mean, dys = Ky_and_derivs_arg(diff_z, diff_t, param, y, prior_mean, fix_boundary)
[dydz, dydt, d2ydz2, d2ydt2] = dys
diff_y = y - post_mean
print("post mean dimension", post_mean.shape)
print("y diff =", np.max(np.abs(diff_y)))

# estimate sink parameter
ny = len(y)
alpha = np.full(ny,5.87)
theta_r = np.full(ny,0.156)
theta_s = np.full(ny,0.60)
m = np.full(ny,0.273)
k_s = np.full(ny,6e-6)

def alpha_func(H, a1, a2, a3, a4):
    if H < a4 or H >= a1:
        return 0.0
    if H >= a4 and H < a3:
        return (H - a4)/(a3 - a4)
    if H >= a3 and H < a2:
        return 1.0
    if H >= a2 and H < a1:
        return (a1 - H)/(a1 - a2)
        
def sink(z, t, h, sink_para, beta, lm) :
    nz = len(z)
    tp = sink_para[:, 0]
    zr = sink_para[:, 1]
    a1 = sink_para[:, 2]
    a2 = sink_para[:, 3]
    a3 = sink_para[:, 4]
    a4 = sink_para[:, 5]
    sink_term = np.zeros(nz)
    for i in range(nz):
        index = int(t[i]/86400 - 1)
        if (abs(z[i]) <= lm[0] * zr[index]):
            alph = alpha_func(h[i] , a1[index], a2[index], a3[index], a4[index])
            fir = (1.0 + beta[0]) * tp[index] / (lm[0] *zr[index]) 
            sink_term[i] = fir * alph * pow(1.0 - abs(z[i]) / (lm[0] * zr[index]), beta[0])
    return sink_term
        
cov = np.loadtxt('cov.txt', skiprows = 1)
sink_para = cov[0:90, 4:10]

######## FIT #########
h = np.vectorize(h_)(post_mean, theta_r, theta_s, 1/(1-m), m, alpha)
s_obs = np.vectorize(S_)(post_mean, theta_r, theta_s)
X_z = X[:, 0]
X_t = X[:, 1]
K = np.vectorize(k_)(h, s_obs, k_s, m)
resid = dydt - (term1(dydz, s_obs, k_s, h, alpha, m, theta_r, theta_s) + K * term2(dydz, d2ydz2, s_obs, h, alpha, m, theta_r, theta_s) + term3(dydz, s_obs, k_s, m, theta_r, theta_s))
# print("resid=",resid)
beta = np.array([2.0])
lm = np.array([1.35])

def sse(lm,beta,  resid, X_z, X_t, h, sink_para,w):
    residfull = resid + sink(X_z, X_t, h, sink_para, beta, lm)
    # w = lm_weight(X_z, X_t, h, sink_para, beta, lm)
    w = w/ np.sum(w)
    return(np.sum(w * residfull ** 2))
w = np.full(len(resid), 1.0)
opt_ini = minimize(fun=sse, x0 =np.array([2.0,1.0]) ,  args= (resid, X_z, X_t, h, sink_para,w),method = 'Nelder-Mead') 
lm_gp = opt_ini.x
print("lm", lm_gp)


# def ssre(par_est,  resid, X_z, X_t, h, sink_para,w):
#     [beta, lm] = par_est
#     beta = np.array([beta])
#     lm = np.array([lm])
#     residfull = resid + sink(X_z, X_t, h, sink_para, beta, lm)
#     # w = beta_weight(X_z, X_t, h, sink_para, beta, lm)
#     w = w/ np.sum(w)
#     return(np.sum(w * residfull ** 2))

# w = np.full(len(resid), 1.0)
# opt_para = minimize(fun=ssre, x0 =np.array([1.5,1.0]) ,  args= (resid, X_z, X_t, h, sink_para,w),method = 'Nelder-Mead') 
# print("par",opt_para.x)
# print("est", opt_para.fun)
# print("true",ssre(np.array([2.0,1.35]), resid, X_z, X_t, h, sink_para,w))


Importing packages...
Importing packages...Done
theta dimension (59, 90)
5310
length of sel 4214
X shape (422, 2)
y length 422
initial log_like = -284.06010078862596
initial deri= [  7.48   -10.141   -9.5051 197.1356]
15:36:25 [-4.2193 -1.5352 14.816  -5.986 ]
15:36:25 [-4.4398 -1.2884 14.741  -6.8647]
15:36:25 [-4.18   -1.761  13.4839 -7.0305]
15:36:26 [-4.0797 -1.9676 12.8302 -7.2364]
15:36:26 [-4.1192 -1.9698 12.4539 -7.4839]
15:36:26 [-4.568  -1.544  12.3615 -8.3098]
15:36:26 [-4.6565 -1.8942 12.6456 -8.5938]
15:36:26 [-4.7184 -2.2619 12.4847 -8.8427]
15:36:26 [-4.8114 -2.1428 12.4337 -8.8546]
15:36:26 [-4.918  -2.1614 12.4377 -8.863 ]
15:36:26 [-6.1249 -2.4445 12.3302 -8.9241]
15:36:26 [-5.7205 -2.3886 12.3669 -8.823 ]
15:36:26 [-5.847  -2.4313 12.3309 -8.8586]
15:36:26 [-5.9052 -2.4495 12.3173 -8.8546]
15:36:26 [-5.8939 -2.4483 12.3162 -8.8589]
15:36:26 [-5.8937 -2.448  12.3167 -8.8576]
15:36:26 [-5.8938 -2.4481 12.3166 -8.8576]
opt para = [2.7565e-03 8.6456e-02 2.2338e+05 1.4229

TypeError: sse() missing 1 required positional argument: 'w'

In [6]:
print("Importing packages...")
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.linalg as la

sys.path.insert(1, 'C:/Users/31676/Desktop/research/pde/Python code/GPPDE_v0/src')
from hydro import *
from gp import *
from funcs import *
from richards import picard_solver
from viz import plot_3d
print("Importing packages...Done")

np.set_printoptions(precision=4)
np.set_printoptions(threshold=np.inf)

thta = np.loadtxt('sol.txt')
print("theta dimension", thta.shape)

dz = -0.005
z = np.arange(-0.01,-0.3+dz,dz)

dt = 86400
t = np.arange(dt, dt * 91, dt)

noise_pct = 0.01
X = np.dstack(np.meshgrid(z, t)).reshape(-1, 2)
y = thta.T.flatten()
y += noise_pct * np.std(y) * np.random.randn(len(y))
h_para = np.loadtxt('para.txt', skiprows = 1)

ny = len(y)
print(ny)
bound_ind = np.array([])
for i in range(ny):
    if(abs(X[i, 0]) <= 0.274) and (abs(X[i, 0]) >= 0.026) and (X[i, 1] > 86400 * 4):
        bound_ind = np.append(bound_ind, i)
print("length of sel", len(bound_ind))
bound_ind = bound_ind[np.arange(0, len(bound_ind), 10)]
bad = bound_ind.astype(int)
X = X[bad, :]
y = y[bad]
    
print("X shape", X.shape)
print("y length", len(y))

Importing packages...
Importing packages...Done
theta dimension (59, 90)
5310
length of sel 4214
X shape (422, 2)
y length 422


In [ ]:
%run sink_select.py

In [1]:
import sys
import numpy as np
sys.path.insert(1, 'C:/Users/31676/Desktop/research/pde/Python code/GPPDE_v0/src')
from hydro import *
from gp import *
from funcs import *
from richards import picard_solver
from viz import plot_3d

thta = np.loadtxt('thta_sink.txt')
dydz = np.loadtxt('dydz_sink.txt')
d2ydz2 = np.loadtxt('d2ydz2_sink.txt')
dydt = np.loadtxt('dydt_sink.txt')
sink1 = np.loadtxt('sink_term.txt')

initi = np.loadtxt('init.txt', skiprows = 1)
z = - np.copy(initi[:, 0])

dt = 86400
t = np.arange(dt, dt * 112, dt)

X = np.dstack(np.meshgrid(z, t)).reshape(-1, 2)
y = thta.T.flatten()
dydz = dydz.T.flatten()
d2ydz2 = d2ydz2.T.flatten()
dydt = dydt.T.flatten()
sink1 = sink1.T.flatten()

def alpha_func(H, a1, a2, a3, a4):
        if H < a4 or H >= a1:
            return 0.0
        if H >= a4 and H < a3:
            return (H - a4)/(a3 - a4)
        if H >= a3 and H < a2:
            return 1.0
        if H >= a2 and H < a1:
            return (a1 - H)/(a1 - a2)
            
def sink(z, t, h, sink_para, beta, lm) :
    nz = len(z)
    tp = sink_para[:, 0]
    zr = sink_para[:, 1]
    a1 = sink_para[:, 2]
    a2 = sink_para[:, 3]
    a3 = sink_para[:, 4]
    a4 = sink_para[:, 5]
    sink_term = np.zeros(nz)
    for i in range(nz):
        index = int(t[i]/86400 - 1)
        if (abs(z[i]) <= lm[0] * zr[index]):
            alph = alpha_func(h[i] , a1[index], a2[index], a3[index], a4[index])
            fir = (1.0 + beta[0]) * tp[index] / (lm[0] *zr[index]) 
            sink_term[i] = fir * alph * pow(1.0 - abs(z[i]) / (lm[0] * zr[index]), beta[0])
    return sink_term
    
cov = np.loadtxt('cov.txt', skiprows = 1)
sink_para = cov[:, 4:10]

nz = len(z)
alpha = np.zeros(nz)
theta_r = np.zeros(nz)
theta_s = np.zeros(nz)
m = np.zeros(nz)
k_s = np.zeros(nz)
h_para = np.loadtxt('para.txt', skiprows = 1)
cur = 0
for i in range(nz):
    if z[i] < -h_para[cur, 0]:
        cur = cur + 1
    theta_r[i] = h_para[cur, 1]
    theta_s[i] = h_para[cur, 2]
    alpha[i] = h_para[cur, 3]
    m[i] = h_para[cur, 4]
    k_s[i] = h_para[cur, 5]

theta_r = np.tile(theta_r, 111)
theta_s = np.tile(theta_s, 111)
alpha = np.tile(alpha, 111)
m = np.tile(m, 111)
k_s = np.tile(k_s, 111)
beta = np.array([2.0])
lm = np.array([1.35])
h = np.vectorize(h_)(y, theta_r, theta_s, 1/(1-m), m, alpha)
s_obs = np.vectorize(S_)(y, theta_r, theta_s)
X_z = X[:, 0]
X_t = X[:, 1]
K = np.vectorize(k_)(h, s_obs, k_s, m)
resid = dydt - (term1(dydz, s_obs, k_s, h, alpha, m, theta_r, theta_s) + K * term2(dydz, d2ydz2, s_obs, h, alpha, m, theta_r, theta_s) + term3(dydz, s_obs, k_s, m, theta_r, theta_s))+ sink1
np.set_printoptions(threshold=np.inf)
print(resid)




[-9.35521992e-07 -9.04287134e-07 -1.08483129e-07 -7.70435362e-09
 -1.02620752e-10 -4.30120800e-08 -3.84667648e-08  8.21269283e-10
  1.31574667e-09 -3.49061421e-08 -2.72449029e-07 -3.17932053e-07
  1.02671649e-05 -5.50760269e-11  7.07268437e-11  2.62769756e-10
  4.36609755e-10  9.96003880e-08  1.05393321e-07  7.90697362e-10
  1.15566362e-10 -2.72829779e-12  1.05590982e-11 -1.73751567e-08
 -1.66660279e-08  5.26565691e-11  2.45121931e-11  6.42689625e-12
 -1.13349988e-13  4.90938887e-09  5.13740216e-09  1.69744532e-12
  2.16770875e-13  6.34347116e-13  5.37934428e-12  9.59325047e-13
  6.19429348e-12  5.12014862e-12  2.02061959e-12  4.89071980e-12
  6.29274134e-12  6.60307936e-13  6.51501685e-12  6.37261503e-13
  8.56169026e-13  1.03829633e-11  3.69136636e-12  1.66016312e-11
  4.36679245e-12  6.69342646e-12  3.65196745e-11  3.83704803e-11
 -5.13067196e-12  3.59102714e-11  6.22024572e-13 -3.28139007e-11
  1.38294241e-10  2.75250593e-11  1.33053404e-12  2.87425166e-14
  3.53390039e-16  2.80317

In [27]:
# resid = dydt - (term1(dydz, s_obs, k_s, h, alpha, m, theta_r, theta_s) + K * term2(dydz, d2ydz2, s_obs, h, alpha, m, theta_r, theta_s) + term3(dydz, s_obs, k_s, m, theta_r, theta_s))
# print(resid)
resid = dydt - (term1(dydz, s_obs, k_s, h, alpha, m, theta_r, theta_s) + K * term2(dydz, d2ydz2, s_obs, h, alpha, m, theta_r, theta_s) + term3(dydz, s_obs, k_s, m, theta_r, theta_s))+ sink1
h1 = np.loadtxt('h.txt')
print(resid)

[-9.35521992e-07 -9.04287134e-07 -1.08483129e-07 -7.70435362e-09
 -1.02620752e-10 -4.30120800e-08 -3.84667648e-08  8.21269283e-10
  1.31574667e-09 -3.49061421e-08 -2.72449029e-07 -3.17932053e-07
  1.02671649e-05 -5.50760269e-11  7.07268437e-11  2.62769756e-10
  4.36609755e-10  9.96003880e-08  1.05393321e-07  7.90697362e-10
  1.15566362e-10 -2.72829779e-12  1.05590982e-11 -1.73751567e-08
 -1.66660279e-08  5.26565691e-11  2.45121931e-11  6.42689625e-12
 -1.13349988e-13  4.90938887e-09  5.13740216e-09  1.69744532e-12
  2.16770875e-13  6.34347116e-13  5.37934428e-12  9.59325047e-13
  6.19429348e-12  5.12014862e-12  2.02061959e-12  4.89071980e-12
  6.29274134e-12  6.60307936e-13  6.51501685e-12  6.37261503e-13
  8.56169026e-13  1.03829633e-11  3.69136636e-12  1.66016312e-11
  4.36679245e-12  6.69342646e-12  3.65196745e-11  3.83704803e-11
 -5.13067196e-12  3.59102714e-11  6.22024572e-13 -3.28139007e-11
  1.38294241e-10  2.75250593e-11  1.33053404e-12  2.87425166e-14
  3.53390039e-16  2.80317

In [12]:
import sys
import numpy as np
sys.path.insert(1, 'C:/Users/31676/Desktop/research/pde/Python code/GPPDE_v0/src')
from hydro import *
from gp import *
from funcs import *
from richards import picard_solver
from viz import plot_3d
dz = -0.025
initi = np.loadtxt('init.txt', skiprows = 1)
z = - np.copy(initi[:, 0])

dt = 86400
t = np.arange(dt, dt * 112, dt)

noise_pct = 0.01
X = np.dstack(np.meshgrid(z, t)).reshape(-1, 2)
diff_z = np.subtract.outer(X[:,0], X[:,0])
diff_t = np.subtract.outer(X[:,1], X[:,1])
np.subtract.outer(np.array([1,2,3]),np.array([1,2,3]))

array([[ 0, -1, -2],
       [ 1,  0, -1],
       [ 2,  1,  0]])